# Légszennyezettség mérés

In [296]:
from __future__ import division
from numpy import genfromtxt
import numpy as np
import csv
import pickle

In [92]:
train_data_file = '../res/pontbeli_adatok_Budapest_dt_sep.csv'

train_data = genfromtxt(train_data_file, delimiter=',', skip_header=1).astype(np.float32)
train_data = np.delete(train_data, -1, 1)
train_data.shape

(23623, 31)

Loading labels + dates to merge data & labels:

In [94]:
train_dates_file = '../res/pontbeli_adatok_Budapest.csv'
train_data_dates = []
with open(train_dates_file, 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    included_cols = [0]
    for row in reader:
        train_data_dates.append(list(row[i] for i in included_cols))
train_data_dates = train_data_dates[1:]

train_labels_file = '../res/Szena_dt_sep.csv'
train_labels_raw = []
with open(train_labels_file, 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    included_cols = [0, 2]
    for row in reader:
        train_labels_raw.append(list(row[i] for i in included_cols))
# Cut header
train_labels_raw = train_labels_raw[1:]

In [111]:
print(len(train_data_dates))
print(len(train_labels_raw))

23623
119249


In [177]:
train_labels_filtered = np.array([i for i in train_labels_raw if i[1] != ''])
t = np.reshape(train_labels_filtered, (-1, 2))
train_labels = t[:, 1].astype(np.float32)
train_labels_dates = t[:, 0]

In [122]:
indices_train_data = np.nonzero(np.in1d(train_data_dates, train_labels_dates))[0]
indices_label = np.nonzero(np.in1d(train_labels_dates, train_data_dates))[0]
print(indices_train_data)
print(indices_label)
print(len(indices_train_data))
print(len(indices_label))

[    0     1     2 ..., 23617 23618 23619]
[ 42579  42582  42585 ..., 108223 108226 108229]
21873
21873


In [121]:
print(train_data_dates[0])
print(train_labels_filtered[42579])

['2009-01-01-00-00']
['2009-01-01-00-00', '67.0']


In [229]:
data_full = train_data[indices_train_data]
label_full_timestamp = train_labels_filtered[indices_label]
label_full = train_labels[indices_label]
label_full = (label_full > 50).astype(np.float32)
cnt_neg = sum(label_full == 0.0)
cnt_pos = sum(label_full == 1.0)
pct_neg = cnt_neg / len(y) * 100
pct_pos = cnt_pos / len(y) * 100
print('Number of 0 labels: %d (%.2f%%)' % (cnt_neg, pct_neg))
print('Number of 1 labels: %d (%.2f%%)' % (cnt_pos, pct_pos))

Number of 0 labels: 17215 (78.70%)
Number of 1 labels: 4658 (21.30%)


In [227]:
data = np.ndarray((data_full.shape[0], data_full.shape[1] + 1))
data[:, 0:-1] = data_full
data[:, data_full.shape[1]] = label_full

array([  2.00900000e+03,   1.00000000e+00,   1.00000000e+00,
         1.20000000e+01,   0.00000000e+00,   1.02563750e+05,
         2.70000005e+00,  -2.59999990e-01,   5.37999992e+01,
        -3.98999643e+00,   5.40440002e+02,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   7.00000000e+01,
         7.84999990e+00,  -4.44000006e+00,  -7.75000620e+00,
         4.80000000e+01,   7.07999992e+00,  -8.22000027e+00,
        -3.84998178e+00,  -2.77752392e-02,   1.36326337e+00,
         5.49558838e+03,   4.62276163e-03,   7.80149102e-02,
         1.02563750e+05,   6.83827475e-02,   1.38646388e+00,
         1.46758704e+03,   0.00000000e+00])

In [211]:
rnd_indeces = np.arange(len(indices_label))
np.random.shuffle(rnd_indeces)

In [262]:
train_data_cnt = int(cnt_pos * 0.8)
valid_data_cnt = int(cnt_pos * 0.1)
test_data_cnt = cnt_pos - train_data_cnt - valid_data_cnt

print('Train data count: %d' % train_data_cnt)
print('Cross validation data count: %d' % valid_data_cnt)
print('Test data count: %d' % test_data_cnt)

Train data count: 3726
Cross validation data count: 465
Test data count: 467


Create train (80%) - validation (10%) - test (rest) datasets.  
Proportion of pos/neg examples in train and valid sets: 50-50  
Proportion of pos/neg examples in test set: 3-97 

In [276]:
data_pos_idx = np.where(label_full == 1)
data_neg_idx = np.where(label_full == 0)

data_pos = data[data_pos_idx]
data_neg = data[data_neg_idx]

train_data_pos = data_pos[0:train_data_cnt]
valid_data_pos = data_pos[train_data_cnt:train_data_cnt+valid_data_cnt]
test_data_pos = data_pos[train_data_cnt+valid_data_cnt:]

train_data_neg = data_neg[0:train_data_cnt]
valid_data_neg = data_neg[train_data_cnt:train_data_cnt+valid_data_cnt]
test_data_neg = data_neg[train_data_cnt+valid_data_cnt:]

print('Pos data counts:')
print(len(train_data_pos))
print(len(valid_data_pos))
print(len(test_data_pos))
print('\nNeg data counts:')
print(len(train_data_neg))
print(len(valid_data_neg))
print(len(test_data_neg))

train_data = np.vstack((train_data_neg, train_data_pos))
valid_data = np.vstack((valid_data_neg, valid_data_pos))
test_data = np.vstack((test_data_neg, test_data_pos))

Pos data counts:
3726
465
467

Neg data counts:
3726
465
13024


Random shuffle:

In [293]:
np.random.shuffle(train_data)
np.random.shuffle(valid_data)
np.random.shuffle(test_data)

print('Neg data counts:')
print(len(train_data))
print(len(valid_data))
print(len(test_data))

Neg data counts:
7452
930
13491


Save pickles:

In [295]:
train_file_name = '../res/input_data/train.pickle'
valid_file_name = '../res/input_data/valid.pickle'
test_file_name = '../res/input_data/test.pickle'

def save_pickle(dataset, set_filename):
    with open(set_filename, 'wb') as f:
        pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
        print('Save {} file'.format(set_filename))
        
save_pickle(train_data, train_file_name)
save_pickle(valid_data, valid_file_name)
save_pickle(test_data, test_file_name)

NameError: global name 'pickle' is not defined